In [13]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import json
from time import sleep
from datetime import datetime

In [14]:
def scrape_page(req):
    
    
    soup = bs(req.content)
    data = soup.find('script', {'id':'__NEXT_DATA__'})
    data = json.loads(data.text)
    data = data['props']['pageProps']['searchPageState']['cat1']['searchResults']['listResults'] # This is one page of results
    
    df = pd.json_normalize(data)

    return df


In [15]:
def scrape_results():
    dfs = []
    headers={
        "accept-language": "en-US,en;q=0.9",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "accept-language": "en-US;en;q=0.9",
        "accept-encoding": "gzip, deflate, br",
    }
    page_num = 1
    url = f'https://www.zillow.com/rockville-md/{page_num}_p/'
    req = requests.get(url, headers=headers)
    
    
    # Messy code - clean it up
    while req.status_code == 200:
        df = scrape_page(req)
        dfs.append(df)
        sleep(10)
        page_num += 1
        
        url = f'https://www.zillow.com/montgomery-county-md/{page_num}_p/'
        req = requests.get(url, headers=headers, allow_redirects=False)

    
    dfs = pd.concat(dfs, axis=0)
    dfs = dfs.drop_duplicates(subset='zpid')
    dfs = dfs.reset_index(drop=True)
    
    # Save the dfs
    dfs.to_csv('data/zillow_raw.csv', index=False)
    
    return dfs

In [16]:
def getNumberFromAddressStreet(addressStreet):
    number =  addressStreet.split(' ')[0]
    # Some houses have no street number
    if number.isnumeric():
        return int(number)
    return np.nan

def getStreetFromAddressStreet(addressStreet):
    addressStreet =  addressStreet.split(' ')
    # Some houses have no street number - return entire addressstreet
    if addressStreet[0].isnumeric():
        return ' '.join(addressStreet[1:])
    return ' '.join(addressStreet)

In [17]:
def renameColumns(df):
        columns = ['hdpData.homeInfo.zpid','addressStreet', 'hdpData.homeInfo.unit', 'addressCity', 'addressState', 'addressZipcode','hdpData.homeInfo.homeType',  'beds', 'baths', 'area', 'pgapt',
                'hdpData.homeInfo.price', 'hdpData.homeInfo.priceForHDP', 'hdpData.homeInfo.taxAssessedValue', 'zestimate', 'hdpData.homeInfo.rentZestimate',
                'hdpData.homeInfo.daysOnZillow','hdpData.homeInfo.isPreforeclosureAuction',  'brokerName', 'latLong.latitude', 'latLong.longitude',  
                'hasOpenHouse', 'openHouseStartDate', 'openHouseEndDate', 'builderName',
                'hdpData.homeInfo.listing_sub_type.is_newHome']

        # Drop prefixes from columns
        df = df[columns]
        columns = [col.split('.')[-1] for col in columns]

        df.columns = columns

        return df

def transformColumns(df):
# Format addresses
        df['streetNumber'] = df['addressStreet'].map(getNumberFromAddressStreet)
        df['street'] = df['addressStreet'].map(getStreetFromAddressStreet)

        # Get streets with no unit name
        streets = []
        for idx, street in enumerate(df['street']):
                unit = df['unit'][idx]
                if type(unit) == str:
                        street = street[:len(street) - len(unit)]
                streets.append(street.strip())

        df['street'] = streets
        df['openHouseStartDate'] = pd.to_datetime(df['openHouseStartDate'])
        df['openHouseEndDate'] = pd.to_datetime(df['openHouseEndDate'])

        # Datetime conversion
        df['openHouseStartDate'] = pd.to_datetime(df['openHouseStartDate'])
        df['openHouseEndDate'] = pd.to_datetime(df['openHouseEndDate'])

        # Some cleaning
        df = df[~df['addressStreet'].str.lower().str.contains('homes available soon')]
        
        return df

def createURLS(df):
        # Create URLs
        propertyURLprefix = 'https://www.zillow.com/homedetails/'
        propertyURLSuffix = df['addressStreet'].str.replace(' ', '-').str.replace('#', '') + '-' + df['addressCity'].str.replace(' ', '-') + '-' + df['addressState'] + '-' + df['addressZipcode'] + '/' + df['zpid'].astype(str) + '_zpid'
        propertyURL = propertyURLprefix + propertyURLSuffix
        df['url'] = propertyURL

        return df

def transformHometype(df):
        df = df[df['homeType'].str.lower() != 'lot']
        df['homeType'] = df['homeType'].map(lambda x: ' '.join(x.split('_')).title())
        df = df.rename({'pgapt':'homeStatus'}, axis=1)
        
        return df

In [36]:
def clean():
    # Load the data from disk
    df = pd. read_csv('data/zillow_raw.csv')
    
    columns = ['hdpData.homeInfo.zpid','addressStreet', 'hdpData.homeInfo.unit', 'addressCity', 'addressState', 'addressZipcode','hdpData.homeInfo.homeType',  'beds', 'baths', 'area', 'pgapt',
            'hdpData.homeInfo.price', 'hdpData.homeInfo.priceForHDP', 'hdpData.homeInfo.taxAssessedValue', 'zestimate', 'hdpData.homeInfo.rentZestimate',
            'hdpData.homeInfo.daysOnZillow','hdpData.homeInfo.isPreforeclosureAuction',  'brokerName', 'latLong.latitude', 'latLong.longitude',  
            'hasOpenHouse', 'openHouseStartDate', 'openHouseEndDate', 'builderName',
            'hdpData.homeInfo.listing_sub_type.is_newHome']

    # Drop prefixes from columns
    df = df[columns]
    columns = [col.split('.')[-1] for col in columns]

    df.columns = columns


    # Format addresses
    df['streetNumber'] = df['addressStreet'].map(getNumberFromAddressStreet)
    df['street'] = df['addressStreet'].map(getStreetFromAddressStreet)

    # Get streets with no unit name
    streets = []
    for idx, street in enumerate(df['street']):
        unit = df['unit'][idx]
        if type(unit) == str:
            street = street[:len(street) - len(unit)]
        streets.append(street.strip())
        
    df['street'] = streets
    df['openHouseStartDate'] = pd.to_datetime(df['openHouseStartDate'])
    df['openHouseEndDate'] = pd.to_datetime(df['openHouseEndDate'])

    # Datetime conversion
    df['openHouseStartDate'] = pd.to_datetime(df['openHouseStartDate'])
    df['openHouseEndDate'] = pd.to_datetime(df['openHouseEndDate'])

    # Some cleaning
    df = df[~df['addressStreet'].str.lower().str.contains('homes available soon')]

    # Create URLs
    df['zpid'] = df['zpid'].astype(str)
    df['addressZipcode'] = df['addressZipcode'].astype(str)
    propertyURLprefix = 'https://www.zillow.com/homedetails/'
    propertyURLSuffix = df['addressStreet'].str.replace(' ', '-').str.replace('#', '') + '-' + df['addressCity'].str.replace(' ', '-') + '-' + df['addressState'] + '-' + df['addressZipcode'] + '/' + df['zpid'].astype(str) + '_zpid'
    propertyURL = propertyURLprefix + propertyURLSuffix
    df['url'] = propertyURL

    df = df[df['homeType'].str.lower() != 'lot']
    df['homeType'] = df['homeType'].map(lambda x: ' '.join(x.split('_')).title())
    
    df = df.rename({'pgapt':'homeStatus'}, axis=1)
    
    # Cleaning
    df['unit'] = df['unit'].replace(np.nan, "No Unit")
    df['brokerName'] = df['brokerName'].replace(np.nan, "Broke Unknown")
    df['hasOpenHouse'] = df['hasOpenHouse'].replace(np.nan, False)
    df['openHouseStartDate'] = df['openHouseStartDate'].replace(pd.NaT, 'No Time')
    df['openHouseEndDate'] = df['openHouseEndDate'].replace(pd.NaT, 'No Time')
    df['builderName'] = df['builderName'].replace(np.nan, 'Builder Unknown')
    df['is_newHome'] = df['is_newHome'].replace(np.nan, False)
    df = df[~df['streetNumber'].isna()]
    # Save the data to disk
    ts = datetime.now().strftime('%m-%d-%Y_%H%M%S')
    
    
    df.to_csv(f'data/zillow_clean_{ts}.csv', index=False)
    
    return df

In [37]:
clean()

,zpid,addressStreet,unit,addressCity,addressState,addressZipcode,homeType,beds,baths,area,...,latitude,longitude,hasOpenHouse,openHouseStartDate,openHouseEndDate,builderName,is_newHome,streetNumber,street,url
0,60528518,221 Jay Dr #102,# 102,Rockville,MD,20850,Townhouse,2.0,3.0,1449.0,...,39.09447,-77.19359,False,No Time,No Time,Builder Unknown,False,221.0,Jay Dr,https://www.zillow.com/homedetails/221-Jay-Dr-...
1,37202483,4601 Sunflower Dr,No Unit,Rockville,MD,20853,Single Family,4.0,3.0,2860.0,...,39.10664,-77.09097,False,No Time,No Time,Builder Unknown,False,4601.0,Sunflower Dr,https://www.zillow.com/homedetails/4601-Sunflo...
2,37100501,304 Baltimore Rd,No Unit,Rockville,MD,20850,Single Family,4.0,3.0,2052.0,...,39.08398,-77.14145,False,No Time,No Time,Builder Unknown,False,304.0,Baltimore Rd,https://www.zillow.com/homedetails/304-Baltimo...
3,58243090,15622 Thistlebridge Dr,No Unit,Rockville,MD,20853,Single Family,5.0,5.0,6489.0,...,39.11601,-77.07990,False,No Time,No Time,Builder Unknown,False,15622.0,Thistlebridge Dr,https://www.zillow.com/homedetails/15622-Thist...
4,37304507,13004 Freeland Rd,No Unit,Rockville,MD,20853,Single Family,3.0,2.0,1470.0,...,39.06936,-77.09025,False,No Time,No Time,Builder Unknown,False,13004.0,Freeland Rd,https://www.zillow.com/homedetails/13004-Freel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,37181432,7516 Westfield Dr,No Unit,Bethesda,MD,20817,Single Family,7.0,9.0,5095.0,...,38.98506,-77.12480,False,No Time,No Time,"Castlewood Custom Builders, Llc",True,7516.0,Westfield Dr,https://www.zillow.com/homedetails/7516-Westfi...
779,241914974,12619 Macaulay St,No Unit,Silver Spring,MD,20906,Townhouse,3.0,5.0,1860.0,...,39.06391,-77.05196,True,2023-10-15 13:00:00,2023-10-15 16:00:00,Builder Unknown,False,12619.0,Macaulay St,https://www.zillow.com/homedetails/12619-Macau...
780,37318736,9306 Harvey Rd,No Unit,Silver Spring,MD,20910,Single Family,5.0,3.0,2641.0,...,39.00838,-77.02567,False,No Time,No Time,Builder Unknown,False,9306.0,Harvey Rd,https://www.zillow.com/homedetails/9306-Harvey...
781,37101940,403 N Horners Ln,No Unit,Rockville,MD,20850,Single Family,8.0,7.0,2610.0,...,39.08907,-77.14314,True,2023-10-15 13:00:00,2023-10-15 15:00:00,Builder Unknown,False,403.0,N Horners Ln,https://www.zillow.com/homedetails/403-N-Horne...


In [19]:
scrape_results()
clean()

,zpid,addressStreet,unit,addressCity,addressState,addressZipcode,homeType,beds,baths,area,...,latitude,longitude,hasOpenHouse,openHouseStartDate,openHouseEndDate,builderName,is_newHome,streetNumber,street,url
0,60528518,221 Jay Dr #102,# 102,Rockville,MD,20850,Townhouse,2.0,3.0,1449.0,...,39.09447,-77.19359,False,No Time,No Time,Builder Unknown,False,221.0,Jay Dr,https://www.zillow.com/homedetails/221-Jay-Dr-...
1,37202483,4601 Sunflower Dr,No Unit,Rockville,MD,20853,Single Family,4.0,3.0,2860.0,...,39.10664,-77.09097,False,No Time,No Time,Builder Unknown,False,4601.0,Sunflower Dr,https://www.zillow.com/homedetails/4601-Sunflo...
2,37100501,304 Baltimore Rd,No Unit,Rockville,MD,20850,Single Family,4.0,3.0,2052.0,...,39.08398,-77.14145,False,No Time,No Time,Builder Unknown,False,304.0,Baltimore Rd,https://www.zillow.com/homedetails/304-Baltimo...
3,58243090,15622 Thistlebridge Dr,No Unit,Rockville,MD,20853,Single Family,5.0,5.0,6489.0,...,39.11601,-77.07990,False,No Time,No Time,Builder Unknown,False,15622.0,Thistlebridge Dr,https://www.zillow.com/homedetails/15622-Thist...
4,37304507,13004 Freeland Rd,No Unit,Rockville,MD,20853,Single Family,3.0,2.0,1470.0,...,39.06936,-77.09025,False,No Time,No Time,Builder Unknown,False,13004.0,Freeland Rd,https://www.zillow.com/homedetails/13004-Freel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,37181432,7516 Westfield Dr,No Unit,Bethesda,MD,20817,Single Family,7.0,9.0,5095.0,...,38.98506,-77.12480,False,No Time,No Time,"Castlewood Custom Builders, Llc",True,7516.0,Westfield Dr,https://www.zillow.com/homedetails/7516-Westfi...
779,241914974,12619 Macaulay St,No Unit,Silver Spring,MD,20906,Townhouse,3.0,5.0,1860.0,...,39.06391,-77.05196,True,2023-10-15 13:00:00,2023-10-15 16:00:00,Builder Unknown,False,12619.0,Macaulay St,https://www.zillow.com/homedetails/12619-Macau...
780,37318736,9306 Harvey Rd,No Unit,Silver Spring,MD,20910,Single Family,5.0,3.0,2641.0,...,39.00838,-77.02567,False,No Time,No Time,Builder Unknown,False,9306.0,Harvey Rd,https://www.zillow.com/homedetails/9306-Harvey...
781,37101940,403 N Horners Ln,No Unit,Rockville,MD,20850,Single Family,8.0,7.0,2610.0,...,39.08907,-77.14314,True,2023-10-15 13:00:00,2023-10-15 15:00:00,Builder Unknown,False,403.0,N Horners Ln,https://www.zillow.com/homedetails/403-N-Horne...


In [20]:
df = pd.read_csv('data/zillow_clean_10-14-2023_105547 - Copy.csv')
# Impute nulls in columns to be dimensions
df['unit'] = df['unit'].replace(np.nan, "No Unit")
df['brokerName'] = df['brokerName'].replace(np.nan, "Broke Unknown")
df['hasOpenHouse'] = df['hasOpenHouse'].replace(np.nan, False)
df['openHouseStartDate'] = df['openHouseStartDate'].replace(pd.NaT, 'No Time')
df['openHouseEndDate'] = df['openHouseEndDate'].replace(pd.NaT, 'No Time')
df['builderName'] = df['builderName'].replace(np.nan, 'Builder Unknown')
df['is_newHome'] = df['is_newHome'].replace(np.nan, False)


In [29]:
d = pd.read_csv('data\zillow_clean_10-14-2023_231618 - Copy.csv')
df[~df['streetNumber'].isna()]

,zpid,addressStreet,unit,addressCity,addressState,addressZipcode,homeType,beds,baths,area,...,latitude,longitude,hasOpenHouse,openHouseStartDate,openHouseEndDate,builderName,is_newHome,streetNumber,street,url
0,60528518,221 Jay Dr #102,# 102,Rockville,MD,20850,Townhouse,2.0,3.0,1449.0,...,39.09447,-77.19359,False,No Time,No Time,Builder Unknown,False,221.0,Jay Dr,https://www.zillow.com/homedetails/221-Jay-Dr-...
1,37202483,4601 Sunflower Dr,No Unit,Rockville,MD,20853,Single Family,4.0,3.0,2860.0,...,39.10664,-77.09097,False,No Time,No Time,Builder Unknown,False,4601.0,Sunflower Dr,https://www.zillow.com/homedetails/4601-Sunflo...
2,37100501,304 Baltimore Rd,No Unit,Rockville,MD,20850,Single Family,4.0,3.0,2052.0,...,39.08398,-77.14145,False,No Time,No Time,Builder Unknown,False,304.0,Baltimore Rd,https://www.zillow.com/homedetails/304-Baltimo...
3,58243090,15622 Thistlebridge Dr,No Unit,Rockville,MD,20853,Single Family,5.0,5.0,6489.0,...,39.11601,-77.07990,False,No Time,No Time,Builder Unknown,False,15622.0,Thistlebridge Dr,https://www.zillow.com/homedetails/15622-Thist...
4,37304507,13004 Freeland Rd,No Unit,Rockville,MD,20853,Single Family,3.0,2.0,1470.0,...,39.06936,-77.09025,False,No Time,No Time,Builder Unknown,False,13004.0,Freeland Rd,https://www.zillow.com/homedetails/13004-Freel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,37181432,7516 Westfield Dr,No Unit,Bethesda,MD,20817,Single Family,7.0,9.0,5095.0,...,38.98506,-77.12480,False,No Time,No Time,"Castlewood Custom Builders, Llc",True,7516.0,Westfield Dr,https://www.zillow.com/homedetails/7516-Westfi...
756,241914974,12619 Macaulay St,No Unit,Silver Spring,MD,20906,Townhouse,3.0,5.0,1860.0,...,39.06391,-77.05196,True,2023-10-15 13:00:00,2023-10-15 16:00:00,Builder Unknown,False,12619.0,Macaulay St,https://www.zillow.com/homedetails/12619-Macau...
757,37318736,9306 Harvey Rd,No Unit,Silver Spring,MD,20910,Single Family,5.0,3.0,2641.0,...,39.00838,-77.02567,False,No Time,No Time,Builder Unknown,False,9306.0,Harvey Rd,https://www.zillow.com/homedetails/9306-Harvey...
758,37101940,403 N Horners Ln,No Unit,Rockville,MD,20850,Single Family,8.0,7.0,2610.0,...,39.08907,-77.14314,True,2023-10-15 13:00:00,2023-10-15 15:00:00,Builder Unknown,False,403.0,N Horners Ln,https://www.zillow.com/homedetails/403-N-Horne...
